## PDS Group 7

In [1]:
import pandas as pd
import numpy as np

pd.set_option("mode.copy_on_write", True)

In [7]:
df = pd.read_csv("threshold.csv")
df.columns

Index(['Buyer State', 'Buyer County', 'Year', 'Total_MME'], dtype='object')

In [ ]:
# check unique states
df["Buyer State"].unique()

array(['FL', 'GA', 'ID', 'MT', 'NC', 'OR', 'SC', 'WA'], dtype=object)

In [4]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import statsmodels.api as sm
from patsy import dmatrix

In [ ]:
fl_states = states = ["FL", "NC", "SC", "GA"]

df_fl = df[df["Buyer State"].isin(fl_states)]